<img src="http://oproject.org/tiki-download_file.php?fileId=8&display&x=450&y=128">
<img src="http://oproject.org/tiki-download_file.php?fileId=25&display&x=452&y=161">
<img src="http://gfif.udea.edu.co/root/tmva/img/tmva_logo.gif">
## Required header files

In [ ]:
#include <cstdlib>
#include <iostream>
#include <map>
#include <string>

#include "TChain.h"
#include "TFile.h"
#include "TTree.h"
#include "TString.h"
#include "TObjString.h"
#include "TSystem.h"
#include "TROOT.h"

#include "TMVA/Factory.h"
#include "TMVA/Tools.h"
#include "TMVA/DataLoader.h"
#include "TMVA/MethodPyAdaBoost.h"
#include "TMVA/MethodPyGTB.h"
#include "TMVA/MethodPyRandomForest.h"

## Defining required objects

In [ ]:
TMVA::Tools::Instance();

TString outfileName( "PyMVAOutput.root" );
TFile* outputFile = TFile::Open( outfileName, "RECREATE" );

TMVA::Factory *factory = new TMVA::Factory( "TMVAClassification", outputFile,
                                               "!V:ROC:Correlations:!Silent:Color:!DrawProgressBar:AnalysisType=Classification" );
   
TMVA::DataLoader *loader=new TMVA::DataLoader("mymc-dataset");
TFile *input = TFile::Open( "MyMCdataset.root");

## Adding variables to Data Loader

In [ ]:
loader->AddVariable( "var0", 'F' );
loader->AddVariable( "var1", 'F' );
loader->AddVariable( "var2", 'F' );
loader->AddVariable( "var3 := var0-var1", 'F' );
loader->AddVariable( "var4 := var0*var2", 'F' );
loader->AddVariable( "var5 := var1+var2", 'F' );

In [ ]:
std::cout << "--- TMVAClassification       : Using input file: " << input->GetName() << std::endl;
   
// --- Register the training and test trees
TTree *tsignal     = (TTree*)input->Get("MyMCSig");
TTree *tbackground = (TTree*)input->Get("MyMCBkg");
   
Double_t signalWeight     = 1.0;
Double_t backgroundWeight = 1.0;
   
loader->AddSignalTree    ( tsignal,     signalWeight     );
loader->AddBackgroundTree( tbackground, backgroundWeight );   
    
TCut mycuts = ""; 
TCut mycutb = ""; 

loader->fSignalWeight=signalWeight;
loader->fBackgroundWeight=backgroundWeight;
loader->fTreeS=tsignal;
loader->fTreeB=tbackground;
loader->PrepareTrainingAndTestTree( mycuts, mycutb,
                                   "nTrain_Signal=3000:nTrain_Background=3000:nTest_Signal=1449:nTest_Background=1449:SplitMode=Random:NormMode=NumEvents:!V" );

## Booking Methods

In [ ]:
//Random Forest
factory->BookMethod(loader,TMVA::Types::kPyRandomForest, "PyRandomForest","!V:NEstimators=10:Criterion=gini:MaxFeatures=auto:MaxDepth=6:MinSamplesLeaf=3:MinWeightFractionLeaf=0:Bootstrap=kTRUE" );

//Gradient Trees Boosted
factory->BookMethod(loader,TMVA::Types::kPyGTB, "PyGTB","!V:NEstimators=10:Loss=deviance:LearningRate=0.1:Subsample=1:MaxDepth=3:MaxFeatures='auto'" );
   
//Adaptative Boost
factory->BookMethod(loader,TMVA::Types::kPyAdaBoost, "PyAdaBoost","!V:NEstimators=10:LearningRate=1.0:Algorithm=SAMME.R" );

## Training the Methods

Available Booking methods with options for RMVA are



In [ ]:
 factory->TrainAllMethods();

## Testing and Evaluating the data

In [ ]:
factory->TestAllMethods();
factory->EvaluateAllMethods();

In [ ]:
// Save the output
outputFile->Close();

std::cout << "==> Wrote root file: " << outputFile->GetName() << std::endl;
std::cout << "==> TMVAClassification is done!" << std::endl;

delete factory;
delete loader;

## Ploting ROC Curve

In [ ]:
TFile *file=TFile::Open("PyMVAOutput.root");
gDirectory->cd("mymc-dataset");

TMVA::plot_efficiencies(file,  2,gDirectory);
c->Draw();